In [49]:
import nltk
import json
import pandas as pd
import re
import gensim
from glob import glob
from gensim import corpora
from nltk.tokenize import word_tokenize

In [56]:
with open("../data/museum/caption2.txt", encoding='UTF8') as f:
    captions=f.readlines()

In [88]:
nltk.download('punkt')
stop_words = ['.','있다.','있는','이','의','를','에','을', '있', '하', '것', '들', '그', '되', 
              '수', '이', '보', '않', '없', '나', '사람', '주', '아니', '등', '같', '우리', '때', '년', 
              '가', '한', '지', '대하', '오', '말', '일', '그렇', '위하', '#', '⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀',
              '더', '다', '울', '앞에', '있다는', '대한', '것을', '나는', '앞에', '내', '✨', '하는',
             '큰', '그런', 'of', 'the']


caption = pd.DataFrame({'caption':captions})
print(caption)
caption['clean_caption'] = caption['caption'].str.replace("^[가-힣]", " ")

tokenized_caption = caption['clean_caption'].apply(lambda x: x.split()) # 토큰화
tokenized_caption = tokenized_caption.apply(lambda x: [item for item in x if item not in stop_words])

dictionary = corpora.Dictionary(tokenized_caption)
corpus = [dictionary.doc2bow(text) for text in tokenized_caption]

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\cho40\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
<ipython-input-88-657ef3140d36>:11: FutureWarning: The default value of regex will change from True to False in a future version.
  caption['clean_caption'] = caption['caption'].str.replace("^[가-힣]", " ")


                                                 caption
0                                   많은 연들이 하늘 높이 떠 있다.\n
1                             흰색과 노란 색의 돋보기이 벽에 걸려 있다.\n
2                                         깃발이 그려진 큰 연단\n
3                                         창문이 있는 방의 사진\n
4                             벽에 걸려 있는 조명 아래에 있는 큰 섬난로\n
...                                                  ...
25019  서울 오늘을 찍다 2019 랜드마크 롯데월드타워  사진전 전시회 서울오늘을찍다 Se...
25020  서울 오늘을 찍다 2019 랜드마크 롯데월드타워  이제 일주일 남았어요 코로나 때문...
25021  drawing drawingfriends 취미 드로잉 드로잉프렌즈 토욜모임 몸은못가...
25022  서울 오늘을 찍다📷  202002110223 여의도 SeMA벙커  재능 있는 친구 ...
25023  비오는 날 여의도 sema 벙커 서울 오늘을 찍다 사진전에서 하루님의 멋진 사진도 ...

[25024 rows x 1 columns]


In [85]:
NUM_TOPICS = 10 #20개의 토픽, k=20
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
topics = ldamodel.print_topics(num_words=10)
for topic in topics:
    print(topic)

(0, '0.026*"에를리치" + 0.025*"레안드로" + 0.021*"그림자를" + 0.019*"드리우고" + 0.013*"색" + 0.012*"Leandro" + 0.011*"레안드로에를리치" + 0.009*"간" + 0.009*"된" + 0.009*"안드로"')
(1, '0.044*"북서울미술관" + 0.035*"레안드로에를리치" + 0.029*"서울시립북서울미술관" + 0.021*"그림자를드리우고" + 0.019*"함께" + 0.016*"사각생각삼각" + 0.013*"서울미술관" + 0.013*"북서울시립미술관" + 0.012*"서울시립" + 0.011*"leandroerlich"')
(2, '0.030*"위한" + 0.018*"물" + 0.015*"옆에" + 0.012*"장소" + 0.009*"하나" + 0.007*"표지판" + 0.007*"앉아" + 0.007*"창문" + 0.006*"붙어" + 0.006*"가구"')
(3, '0.036*"볼" + 0.017*"있고" + 0.014*"그림이" + 0.012*"걸려" + 0.008*"개의" + 0.007*"몇" + 0.007*"그려진" + 0.006*"아이들이" + 0.006*"오래" + 0.006*"고양이"')
(4, '0.055*"°" + 0.035*"같은" + 0.013*"속에서" + 0.008*"가진" + 0.007*"보며" + 0.006*"관한" + 0.006*"거울을" + 0.006*"사람들의" + 0.004*"가까운" + 0.004*"이징월드"')
(5, '0.019*"그림" + 0.013*"다양한" + 0.013*"그의" + 0.012*"시" + 0.010*"것이" + 0.009*"무료" + 0.009*"이야기를" + 0.007*"가지고" + 0.007*"아트" + 0.007*"자신의"')
(6, '0.061*"_" + 0.017*"본" + 0.011*"평창동" + 0.009*"print" + 0.009*"bakery" + 0.008*"KCDF" + 0.008*"공예" + 0.008*

In [64]:
# vector = ldamodel[test_doc]
# print(vector)

[(0, 0.41999856), (1, 0.020000657), (2, 0.020000083), (3, 0.020000102), (4, 0.020000648), (5, 0.41999966), (6, 0.020000083), (7, 0.020000083), (8, 0.020000083), (9, 0.020000083)]


In [65]:
test_doc = corpus[-1]
vector = ldamodel[test_doc]
ldamodel.save('../data/museum/model_topic/test_model.model')

In [66]:
new_model = gensim.models.ldamodel.LdaModel.load("../data/museum/model_topic/test_model.model")

In [87]:
print(corpus[-1])

[(99, 1), (357, 1), (448, 1), (1187, 1), (1511, 1), (1651, 1), (4344, 1), (4484, 1), (5029, 1), (5361, 1), (6940, 1), (9398, 1), (12767, 1), (14980, 1), (15350, 1), (17573, 1), (19120, 1), (29718, 1), (30858, 1), (31714, 1), (43017, 1), (60415, 1), (112984, 1), (112987, 1), (113009, 1), (113010, 1), (113011, 1), (113012, 1), (113013, 1), (113014, 1)]
